In [ ]:
pip install keras_preprocessing

In [ ]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, MaxPooling2D
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## change path once location is changed
#this loads the dataset for training
FACES_DIR= '/content/drive/MyDrive/newtrial/train'
TFACES_DIR ='/content/drive/MyDrive/newtrial/test'

In [ ]:
#to read all images in the dataset
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [ ]:
train= pd.DataFrame()
train['image'], train['label'] = createdataframe(FACES_DIR)

In [ ]:
print(train)

In [ ]:
test =pd.DataFrame()
test['image'],test['label']= createdataframe(TFACES_DIR)

In [ ]:
print (test)

In [ ]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        try:
            img = load_img(image, grayscale=True, target_size=(256, 256))
            img = np.array(img)
            features.append(img)
        except Exception as e:
            print(f"Error loading image: {image}. Error: {str(e)}")

    features = np.array(features)
    features = features.reshape(len(features), 256, 256, 1)
    print(f"Features shape: {features.shape}")
    return features

In [ ]:
train_features = extract_features(train['image'])

In [ ]:
test_features = extract_features(test['image'])

In [ ]:
x_train = train_features/255.0
x_test = test_features/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
le.fit(train['label'])

In [ ]:
y_train = le.transform(train["label"])
y_test = le.transform(test['label'])

In [ ]:
y_train = to_categorical(y_train, num_classes = 2)
y_test =to_categorical(y_test, num_classes = 2)

In [ ]:
from keras.layers import Flatten
from keras import optimizers

In [ ]:
model = Sequential()
#convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', strides=(1,1),padding= 'same', input_shape=(256,256,1)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3),padding= 'same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3),padding= 'same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3),padding= 'same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(Dropout(0.4))

model.add(Flatten())

#fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))

#output layer
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(optimizer= 'adam', loss= 'categorical_crossentropy', metrics= 'accuracy')
model.summary()

In [ ]:
hist = model.fit(x= x_train, y= y_train,batch_size= 32, epochs= 100, validation_data = (x_test, y_test))

In [ ]:
#plot performance
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], 'orange', label= 'val_loss')
fig.suptitle('loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], 'orange', label= 'val_accuracy')
fig.suptitle('accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
model_json = model.to_json()
with open("trainedbatchf2.json", 'w') as json_file:
    json_file.write(model_json)
model.save("trainedbatchf2.h5")

In [ ]:
from keras.models import model_from_json

In [ ]:
json_file = open("trainedbatchf2.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("trainedbatchf2.h5")

In [ ]:
label = ['drunk', 'sober']

In [ ]:
def ef(image):
    img = load_img(image, grayscale = True)
    feature = np.array(img)
    feature =feature.reshape(1,256,256,1)
    return feature/255.0

In [ ]:
image = 'drive/MyDrive/newtrial/verify/cls.jpg'
print("original image is of sober")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

In [ ]:
image = 'drive/MyDrive/newtrial/verify/pst.jpg'
print("original image is of drunk")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)